In [1]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [2]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert data science."),
    ("user", "{input}")
])

chain = prompt | llm 

In [3]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [4]:
chain = prompt | llm | output_parser

In [5]:
chain.invoke({"input": "define data"})

'\nAs an expert in data science, I can tell you that data refers to the information or facts that are collected, stored, and analyzed to gain insights and make informed decisions. Data can come in various forms, including:\n\n1. Numbers: Quantitative data is numerical data that can be measured and represented using numbers, such as sales figures, customer demographics, or stock prices.\n2. Words: Qualitative data is non-numerical information that can provide context and meaning to the quantitative data, such as customer feedback, survey responses, or text data from social media platforms.\n3. Images: Visual data is information that is represented using images, such as photos, videos, or graphics.\n4. Time-series data: This type of data is collected over time and can be used to analyze patterns and trends, such as stock prices, website traffic, or sensor readings.\n5. Sensor data: Data collected from sensors, such as temperature, humidity, or GPS location, can provide valuable insights 

In [48]:
from langchain_community.embeddings import OllamaEmbeddings
import os, json
import pandas as pd
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_community.document_loaders import JSONLoader

file_path='jsons/01-02-2023.json'
data = json.loads(Path(file_path).read_text())
loader = JSONLoader(
         file_path=file_path,
         jq_schema='.[]',
         text_content=False)

docs = loader.load()
print("dados:",docs[3].page_content[:])
print(docs[0].metadata)

#data = pd.read_csv("data.csv")
#print(data[10].page_content[:])

embeddings = OllamaEmbeddings()

dados: {"DATA": "01-02-2023", "CADERNO": 1, "PAGINA": 1, "NOME": "PODER EXECUTIVO", "PUBLICACAO": 4, "TEXTO": "DECRETO N\u00ba35.296, de 31 de janeiro de 2023.REDENOMINA O CENTRO DE FORMA\u00c7\u00c3O DE PROFESSORES DO ESTADO MARIA NELI SOBREIRA DE OLIVEIRA, QUE PASSA A DENOMINAR-SE CENTRO DE FORMA\u00c7\u00c3O E DESENVOLVIMENTO PARA OS PROFISSIONAIS DA EDUCA\u00c7\u00c3O DO ESTADO DO CEAR\u00c1 PROF\u00aa. MARIA NELI SOBREIRA DE OLIVEIRA \u2013 FORMACE, NO MUNIC\u00cdPIO DE FORTALEZA/CE E D\u00c1 OUTRAS PROVID\u00caNCIAS.O GOVERNADOR DO ESTADO DO CEAR\u00c1, no uso das atribui\u00e7\u00f5es que lhe confere o Art. 88, incisos IV e VI, da Constitui\u00e7\u00e3o do Estado e CONSIDERANDO o Art. 84, incisos I a XIII, do Decreto n\u00ba 35.048, de 14 de dezembro de 2022 e CONSIDERANDO a necessidade de atender \u00e0 comunidade escolar, DECRETA:Art.1\u00ba Fica redenominado, na estrutura organizacional da Secretaria da Educa\u00e7\u00e3o do Estado do Cear\u00e1, o CENTRO DE FORMA\u00c7\u00c3

In [51]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [52]:
#geração de regex genericas com agrupamento de queries

from langchain.chains import create_retrieval_chain

from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [56]:
from langchain_core.documents import Document

print(document_chain.invoke({
    "input": "Quais insights é possível retornar a partir desses dados?",
    "context": [Document(page_content="Esse é um banco de dados a respeito do governo")]
}))


Based on the given context, it is possible to gain several insights from the database related to government:

1. Government spending patterns: By analyzing the database, you can identify how much money is being spent by the government on different sectors such as healthcare, education, defense, and infrastructure. This could help in understanding the government's priorities and allocation of resources.
2. Political trends: The database may contain information on political parties, their ideologies, and their performance in elections. This could provide insights into the political landscape and how it has evolved over time.
3. Public opinion surveys: The database may include surveys or polls that gauge public opinion on various issues related to government policies, such as crime rates, taxation, and public safety. This could help in understanding what the general population thinks about the government's performance and priorities.
4. Government efficiency: By analyzing the database, y

In [55]:
response = retrieval_chain.invoke({"input": "Qual o conteúdo desse banco de dados?"})
print(response["answer"])


O banco de dados que você forneceu é um extrato de um processo de alteração de preços dos itens 09 e 10, relativos a uma geladeira com capacidade de 260 litros, além de outros itens como bebedouro coluna, filtro purificador, etc.

O extrato contém informações sobre a data do protocolo (27/06/2022), a data da assinatura (20/12/2022) e a vigência do extrato (a partir da data do protocolo). Além disso, está incluído o nome do responsável pela gestão do processo (Adriano Sarquis Bezerra de Menezes), o representante legal da empresa (Guilherme Henrique da Silva Brand\u00e3o) e o orientador da c\u00c9ula de gest\u00c3o de registro de preços (Soraya Quixad\u00e1 Bezerra).

O extrato também contém informações sobre os itens que são objeto da alteração de preços, incluindo o nome dos itens, o preço original e o novo preço. Por exemplo, os itens 09 e 10 têm o preço original de R$ 1.533,80 e o novo preço de R$ 1.890,00.

Em geral, este banco de dados representa uma situação de alteração de preço

In [57]:
response = retrieval_chain.invoke({"input": "Qual a idéia geral a respeito desse banco de dados?"})
print(response["answer"])


O banco de dados apresentado é uma resolução do Gabinete do Controle Geral de Disciplina, do Estado do Ceará, no Brasil. É um documento que estabelece a instauração de um processo administrativo disciplinar contra o auxilier de percepção Diones Gomes dos Santos, por uma conduta atentatória aos direitos humanos fundamentais e desonrosa, não havendo sido condenado em outros infrações disciplinares nos últimos 5 anos.

A resolução destaca que a tramitação prioritária dos procedimentos administrativos disciplinares envolvendo vítimas de violência doméstica disciplinar está sendo tratada pela Portaria CGD N\u00ba404/2022, publicada no DOE N\u00ba176 em 30 de agosto de 2022. Além disso, é estabelecido que o processo disciplinar será instalado em uma comissão permanente formada pelos delegados de Polícia Civil Milena Martins Monteiro, Rafael Bezerra Cardoso e Escrivão de Polícia Civil Cleodon Pereira Nobre Júnior.

No geral, a resolução estabelece as seguintes decisões:

1. Instauration do p

In [12]:
import { ChatOllama } from "@langchain/community/chat_models/ollama";

const chatModel = new ChatOllama({
  baseUrl: "http://localhost:11434", // Default value
  model: "mistral",
});

SyntaxError: invalid syntax (2021891032.py, line 1)